In [1]:
# Load packages
import IPython.display as ipd
import os
import json
import random
import pydub
import numpy as np

# Import 
from utils import AudioDataset, PostProcess
data_dir = '../../data/audio/kore/'

In [2]:
# Create a list of paths
audio_dir = os.path.join(data_dir,'word_audio_npy')
target_dir = os.path.join(data_dir,'targets')

from glob import glob
audio_dir = os.path.join(data_dir,'word_audio_npy')
glob_pattern = os.path.join(audio_dir, '*')
audio_list = sorted(glob(glob_pattern), key=os.path.getctime)

target_dir = os.path.join(data_dir,'targets')
glob_pattern = os.path.join(target_dir, '*')
target_list = sorted(glob(glob_pattern), key=os.path.getctime)

In [4]:
n_audio_max = 80000
n_target_max = 9
dataset = AudioDataset(audio_list,target_list,n_audio_max,n_target_max)
lookup_dict = json.load(open('./lookup.json'))
postprocessor = PostProcess(lookup_dict)
random_integer = random.randint(0,len(audio_list))
original_word_audio_dir =  os.path.join(data_dir,'kore-sound-vocab-munged')

random_audio, random_target = dataset[random_integer]
random_audio_ID = os.path.split(audio_list[random_integer])[-1][:-4]
original_audio_path = os.path.join(original_word_audio_dir,random_audio_ID) + '.mp3'

rs = int(pydub.utils.mediainfo(original_audio_path)['sample_rate'])


print(postprocessor.target2kana(random_target.numpy()))
ipd.display(ipd.Audio(random_audio.numpy()*2**15, rate=rs,autoplay=True))

オモイガケナイ


In [6]:
n_dataset = len(audio_list)
train_proportion = .9
n_train = int(.9*n_dataset)
inds = np.arange(n_dataset)
np.random.shuffle(inds)
inds_train = inds[:n_train]
inds_val = inds[n_train:]

audio_list_train = np.array(audio_list)[inds_train].tolist()
audio_list_val = np.array(audio_list)[inds_val].tolist()

target_list_train = np.array(target_list)[inds_train].tolist()
target_list_val = np.array(target_list)[inds_val].tolist()

# I know that it possible to use the Subset class to separate the train and validation loaders,
# however I don't know how to make the validation dataset not randomly subsampled in that case

dataset_train = AudioDataset(audio_list_train,target_list_train,n_audio_max,n_target_max)
dataset_val = AudioDataset(audio_list_val,target_list_val,n_audio_max,n_target_max)

batch_size_train = 16
batch_size_val = 32

from torch.utils.data import DataLoader

train_loader = DataLoader(dataset_train, batch_size=batch_size_train,shuffle=True)
val_loader = DataLoader(dataset_val, batch_size=batch_size_val)



In [7]:
for batch in train_loader:
    
    
    
    print(batch)

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 42642 and 48402 in dimension 1 at /opt/conda/conda-bld/pytorch_1565272271120/work/aten/src/TH/generic/THTensor.cpp:689